***DeadLine - 29.09.2024 23:59***


Примечание:

Я убрал инструкцию, потому что мешает. 

С ноутбуком я работаю локально поэтому установка зависимостей ниже.

In [1]:
packages = [
    "jupyter==1.0.0",
    "folium==0.17.0",
    "nltk==3.9.1",
    "gensim==4.3.3",
    "bokeh==3.6.0",
    "matplotlib==3.9.2",
    "seaborn==0.13.2",
    "scikit-learn==1.5.2",
]

with open("requirements.txt", "w") as file:
    file.write("\n".join(packages))

In [2]:
!pip3 install -r requirements.txt > /dev/null

## Семинар 1: Веселье с векторными представлениями слов (2 балла)

Сегодня мы будем играть с векторными представлениями слов: обучим собственные небольшие эмбеддинги, загрузим готовую модель из gensim model zoo и используем её для визуализации текстового корпуса.

Вся работа будет происходить на наборе данных с векторными представлениями.

__Требования:__  `pip install --upgrade nltk gensim bokeh`, но только если вы запускаете код локально.

In [3]:
!wget "https://www.dropbox.com/s/obaitrix9jyu84r/quora.txt?dl=1" -O ./quora.txt

--2024-10-05 17:37:02--  https://www.dropbox.com/s/obaitrix9jyu84r/quora.txt?dl=1
Resolving www.dropbox.com (www.dropbox.com)... 162.125.66.18
Connecting to www.dropbox.com (www.dropbox.com)|162.125.66.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://www.dropbox.com/scl/fi/p0t2dw6oqs6oxpd6zz534/quora.txt?rlkey=bjupppwua4zmd4elz8octecy9&dl=1 [following]
--2024-10-05 17:37:03--  https://www.dropbox.com/scl/fi/p0t2dw6oqs6oxpd6zz534/quora.txt?rlkey=bjupppwua4zmd4elz8octecy9&dl=1
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://ucd51ac30fb65e9c3f4816108eda.dl.dropboxusercontent.com/cd/0/inline/Cb7Yq55vYlF_MrGKZTitiiaPRhzoRnsLf0afS3I3DxzbdnreXsflVPV0xtNUWrXT8oIETmNMzct6gOKjNO_rE-HeBaHx0S5ehE08z8EFHWQhwpDfFU3EgagP9K_Z7qA418w/file?dl=1# [following]
--2024-10-05 17:37:04--  https://ucd51ac30fb65e9c3f4816108eda.dl.dropboxusercontent.com/cd/0/inline/Cb7Yq55vYlF_MrGKZTitiiaPRhzoRnsLf0afS

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

sns.set(style="darkgrid")

np.random.seed(0)

In [5]:
with open("./quora.txt", encoding="utf-8") as file:
    data = list(file)

data[50]

"What TV shows or books help you read people's body language?\n"

__Токенизация:__ типичный первый шаг для задачи обработки естественного языка — это разделение исходных данных на слова. Текст, с которым мы работаем, находится в сыром формате: пунктация, смайлики и прочее

Давайте воспользуемся __`nltk`__ — библиотекой, которая выполняет многие задачи NLP, такие как токенизация, стемминг или определение частей речи.


In [6]:
from nltk.tokenize import WordPunctTokenizer

tokenizer = WordPunctTokenizer()

print(tokenizer.tokenize(data[50]))

['What', 'TV', 'shows', 'or', 'books', 'help', 'you', 'read', 'people', "'", 's', 'body', 'language', '?']


In [7]:
# TASK: lowercase everything and extract tokens with tokenizer.
# data_tok should be a list of lists of tokens for each line in data.

data_tok = list(map(lambda x: tokenizer.tokenize(x.lower()), data)) # map works better 
data_tok[:2]

[['can',
  'i',
  'get',
  'back',
  'with',
  'my',
  'ex',
  'even',
  'though',
  'she',
  'is',
  'pregnant',
  'with',
  'another',
  'guy',
  "'",
  's',
  'baby',
  '?'],
 ['what',
  'are',
  'some',
  'ways',
  'to',
  'overcome',
  'a',
  'fast',
  'food',
  'addiction',
  '?']]

In [8]:
assert all(isinstance(row, (list, tuple)) for row in data_tok), "please convert each line into a list of tokens (strings)"

assert all(all(isinstance(tok, str) for tok in row) for row in data_tok), "please convert each line into a list of tokens (strings)"

is_latin = lambda tok: all('a' <= x.lower() <= 'z' for x in tok)

assert all(map(lambda l: not is_latin(l) or l.islower(), map(' '.join, data_tok))), "please make sure to lowercase the data"

In [9]:
print([' '.join(row) for row in data_tok[:2]])

["can i get back with my ex even though she is pregnant with another guy ' s baby ?", 'what are some ways to overcome a fast food addiction ?']


__Векторы слов:__ как известно, существует множество способов для обучения векторных представлений слов. Есть Word2Vec и GloVe с разными целевыми функциями. Затем есть fastText, который использует n-gram модели для обучения векторных представлений слов.

Выбор огромен, так что давайте начнем с чего-то простого: __gensim__ – еще одна библиотека для обработки естественного языка, включающая множество моделей на основе векторов, включая word2vec.

In [10]:
from gensim.models import Word2Vec

model = Word2Vec(data_tok,
                 vector_size=32, # embedding vector size
                 min_count=5,  # consider words that occured at least 5 times
                 window=5).wv  # define context as a 5-word window around the target word

In [11]:
# now you can get word vectors !
model.get_vector('anything')

array([-2.9718468 ,  0.7667287 ,  1.146965  ,  3.5931923 ,  1.5332683 ,
        0.94223094,  0.9820101 , -4.890552  ,  0.20565215,  2.2298172 ,
        0.49317983,  1.304699  ,  3.7768197 , -0.31017315,  1.4775383 ,
       -1.1749489 ,  0.0674892 , -1.4096224 ,  0.834447  , -2.343525  ,
       -2.2209842 , -0.9681214 , -2.9530983 , -3.2903259 ,  2.8657289 ,
       -2.2350638 , -0.6263929 ,  1.5306731 ,  0.82061917,  1.1886826 ,
        0.32220855,  1.0939745 ], dtype=float32)

In [12]:
# or query similar words directly. Go play with it!
model.most_similar('bread')

[('rice', 0.9576563239097595),
 ('cheese', 0.9403374791145325),
 ('sauce', 0.936443030834198),
 ('butter', 0.9354627728462219),
 ('beans', 0.9251757264137268),
 ('wine', 0.9186714291572571),
 ('chocolate', 0.9160171747207642),
 ('corn', 0.9104917645454407),
 ('cream', 0.9048624634742737),
 ('banana', 0.9048367142677307)]

### Использование предобученной модели

Заняло некоторое время, да? Теперь представьте себе обучение полноразмерных (100~300 измерений) векторных представлений слов на гигабайтах текста: статьи из Википедии или посты в Твиттере.

К счастью, сегодня вы можете получить предобученную модель векторных представлений слов всего за 2 строки кода (без смс и регистрации, обещаю).

In [13]:
import gensim.downloader as gensim_api
model = gensim_api.load('glove-twitter-100')

In [14]:
model.most_similar(positive=["coder", "money"], negative=["brain"])

[('broker', 0.5820155739784241),
 ('bonuses', 0.5424473881721497),
 ('banker', 0.5385112762451172),
 ('designer', 0.5197198390960693),
 ('merchandising', 0.4964233338832855),
 ('treet', 0.49220189452171326),
 ('shopper', 0.4920561909675598),
 ('part-time', 0.4912828207015991),
 ('freelance', 0.4843311905860901),
 ('aupair', 0.4796452522277832)]

### Визуализация векторов слов

Один из способов понять, насколько хороши наши векторы, — это их визуализировать. Проблема в том, что эти векторы находятся в пространстве с размерностью 30+ измерений, а мы, люди, привыкли к 2-3 измерениям.

К счастью, мы, специалисты по машинному обучению, знаем о методах __снижения размерности__.

Давайте используем их, чтобы построить график для 1000 самых частотных слов.

In [15]:
words = model.index_to_key[:1000]

print(words[::100])

['<user>', '_', 'please', 'apa', 'justin', 'text', 'hari', 'playing', 'once', 'sei']


In [16]:
# for each word, compute it's vector with model
word_vectors = np.array(list(map(model.get_vector, words)))

In [17]:
word_vectors

array([[ 0.63006 ,  0.65177 ,  0.25545 , ...,  0.55096 ,  0.64706 ,
        -0.6093  ],
       [ 0.18205 , -0.048483,  0.23966 , ..., -0.3358  ,  0.18884 ,
        -0.40786 ],
       [ 1.0674  ,  0.45716 ,  0.51463 , ...,  0.13974 ,  0.76487 ,
        -0.17314 ],
       ...,
       [ 0.11065 , -0.040876,  0.2586  , ..., -0.74773 ,  0.306   ,
         0.37911 ],
       [ 0.30491 , -0.89558 , -0.46538 , ...,  0.73649 ,  1.3842  ,
         0.99976 ],
       [-0.43051 ,  0.56302 ,  0.38305 , ..., -0.3222  ,  0.32782 ,
         0.51519 ]], dtype=float32)

In [18]:
assert isinstance(word_vectors, np.ndarray)
assert word_vectors.shape == (len(words), 100)
assert np.isfinite(word_vectors).all()

#### Линейная проекция: PCA

Самый простой метод линейного снижения размерности — это Principial Component Analysis.

В геометрических терминах PCA пытается найти оси, вдоль которых происходит наибольшая дисперсия. Это можно назвать "естественными" осями.

<img src="https://github.com/yandexdataschool/Practical_RL/raw/master/yet_another_week/_resource/pca_fish.png" style="width:30%">

Под капотом алгоритм пытается разложить матрицу объект-признак $X$ на две меньшие матрицы: $W$ и $\hat W$, минимизируя _среднеквадратичную ошибку_:

$$\|(X W) \hat{W} - X\|^2_2 \to_{W, \hat{W}} \min$$
- $X \in \mathbb{R}^{n \times m}$ — матрица объектов (**централизованная**);
- $W \in \mathbb{R}^{m \times d}$ — матрица прямого преобразования;
- $\hat{W} \in \mathbb{Р}^{d \times m}$ — матрица обратного преобразования;
- $n$ — количество объектов, $m$ — исходная размерность, $d$ — целевая размерность.

PCA работает, находя такие линейные проекции данных, которые сохраняют максимальное количество информации, измеряемое через дисперсию. Основная идея заключается в том, чтобы найти несколько новых осей (компонент), которые захватывают основную часть структуры данных.

In [19]:
from sklearn.decomposition import PCA
import numpy as np

# map word vectors onto 2d plane with PCA. Use good old sklearn api (fit, transform)
# after that, normalize vectors to make sure they have zero mean and unit variance
word_vectors_pca = PCA(n_components=2).fit_transform(word_vectors)

word_vectors_pca = (word_vectors_pca - word_vectors_pca.mean(axis=0)) / word_vectors_pca.std(axis=0) 

# and maybe MORE OF YOUR CODE here :)

In [20]:
assert word_vectors_pca.shape == (len(word_vectors), 2), "there must be a 2d vector for each word"
assert max(abs(word_vectors_pca.mean(0))) < 1e-5, "points must be zero-centered"
assert max(abs(1.0 - word_vectors_pca.std(0))) < 1e-2, "points must have unit variance"

#### Let's draw it!

In [21]:
import bokeh.models as bm, bokeh.plotting as pl
from bokeh.io import output_notebook
output_notebook()

def draw_vectors(x, y, radius=10, alpha=0.25, color='blue',
                 width=600, height=400, show=True, **kwargs):
    """ draws an interactive plot for data points with auxilirary info on hover """
    if isinstance(color, str): color = [color] * len(x)
    data_source = bm.ColumnDataSource({ 'x' : x, 'y' : y, 'color': color, **kwargs })

    fig = pl.figure(active_scroll='wheel_zoom', width=width, height=height)
    fig.scatter('x', 'y', size=radius, color='color', alpha=alpha, source=data_source)

    fig.add_tools(bm.HoverTool(tooltips=[(key, "@" + key) for key in kwargs.keys()]))
    if show: pl.show(fig)
    return fig

Loading BokehJS ...

In [22]:
draw_vectors(word_vectors_pca[:, 0], word_vectors_pca[:, 1], token=words)

# hover a mouse over there and see if you can identify the clusters

figure(id='p1004', ...)

### Визуализация соседей с помощью t-SNE
PCA хорош для анализа данных, но это строго линейный метод, и он способен улавливать только грубую высокоуровневую структуру данных.

Если же мы хотим сосредоточиться на том, чтобы соседние точки данных оставались рядом, можно использовать t-SNE, который сам по себе является методом векторизации (embedding). Здесь можно __[узнать больше о t-SNE](https://distill.pub/2016/misread-tsne/)__.

t-SNE (t-distributed Stochastic Neighbor Embedding) — это метод нелинейного снижения размерности, который лучше всего подходит для визуализации высокоразмерных данных. Он работает, моделируя вероятность того, что объекты будут соседями в исходном пространстве, и затем пытается сохранить эти вероятности в проекции меньшей размерности. Основная идея t-SNE — это сохранение локальных структур данных, что делает его полезным для кластеризации и анализа сложных структур.

In [23]:
from sklearn.manifold import TSNE

# map word vectors onto 2d plane with TSNE. hint: don't panic it may take a minute or two to fit.
# normalize them as just lke with pca


word_tsne = TSNE(n_components=2).fit_transform(word_vectors)
word_tsne = (word_tsne - word_tsne.mean(axis=0)) / word_tsne.std(axis=0)

In [24]:
draw_vectors(word_tsne[:, 0], word_tsne[:, 1], color='green', token=words)

figure(id='p1055', ...)

### Визуализация фраз

Векторные представления слов также можно использовать для представления коротких фраз. Самый простой способ — взять __среднее значение__ векторов всех токенов в фразе с определёнными весами.

Этот приём полезен для понимания, с какими данными вы работаете: можно определить наличие выбросов, кластеров или других артефактов.

In [25]:
model.has_index_for("hi"), model.has_index_for("hidddwa")

(True, False)

In [26]:
def get_phrase_embedding(phrase: str, show_debug: bool = True) -> np.ndarray:
    """
    Convert phrase to a vector by aggregating it's word embeddings. See description above.
    """
    # 1. lowercase phrase
    # 2. tokenize phrase
    # 3. average word vectors for all words in tokenized phrase
    # skip words that are not in model's vocabulary
    # if all words are missing from vocabulary, return zeros
    
    tokens = tokenizer.tokenize(phrase.lower())
    if show_debug:
        print(f"[get_phrase_embedding] Tokens before filter: {len(tokens)}")
    tokens_filtered = [token for token in tokens if model.has_index_for(token)]
    if show_debug:
        print(f"[get_phrase_embedding] Tokens after filter: {len(tokens_filtered)}")
    if len(tokens_filtered) == 0:
        return np.zeros(shape=model.vector_size)
    
    tokens_filtered = np.array(list(map(model.get_vector, tokens_filtered)))
    
    return tokens_filtered.mean(axis=0)



In [27]:
vector = get_phrase_embedding("I'm very sure. This never happened to me before...")

assert np.allclose(vector[::10],
                   np.array([ 0.31807372, -0.02558171,  0.0933293 , -0.1002182 , -1.0278689 ,
                             -0.16621883,  0.05083408,  0.17989802,  1.3701859 ,  0.08655966],
                              dtype=np.float32))

[get_phrase_embedding] Tokens before filter: 13
[get_phrase_embedding] Tokens after filter: 12


In [28]:
len(data)

537272

In [29]:
# let's only consider ~5k phrases for a first run.
chosen_phrases = data[::(len(data) // 1000)]

# compute vectors for chosen phrases
phrase_vectors = np.array(list(map(get_phrase_embedding, chosen_phrases))) # YOUR CODE

[get_phrase_embedding] Tokens before filter: 19
[get_phrase_embedding] Tokens after filter: 19
[get_phrase_embedding] Tokens before filter: 16
[get_phrase_embedding] Tokens after filter: 16
[get_phrase_embedding] Tokens before filter: 34
[get_phrase_embedding] Tokens after filter: 33
[get_phrase_embedding] Tokens before filter: 12
[get_phrase_embedding] Tokens after filter: 12
[get_phrase_embedding] Tokens before filter: 9
[get_phrase_embedding] Tokens after filter: 9
[get_phrase_embedding] Tokens before filter: 12
[get_phrase_embedding] Tokens after filter: 12
[get_phrase_embedding] Tokens before filter: 18
[get_phrase_embedding] Tokens after filter: 18
[get_phrase_embedding] Tokens before filter: 8
[get_phrase_embedding] Tokens after filter: 7
[get_phrase_embedding] Tokens before filter: 10
[get_phrase_embedding] Tokens after filter: 10
[get_phrase_embedding] Tokens before filter: 9
[get_phrase_embedding] Tokens after filter: 9
[get_phrase_embedding] Tokens before filter: 6
[get_phra

In [30]:
assert isinstance(phrase_vectors, np.ndarray) and np.isfinite(phrase_vectors).all()
assert phrase_vectors.shape == (len(chosen_phrases), model.vector_size)

In [31]:
# map vectors into 2d space with pca, tsne or your other method of choice
# don't forget to normalize

phrase_vectors_2d = PCA(n_components=2).fit_transform(phrase_vectors)

phrase_vectors_2d = (phrase_vectors_2d - phrase_vectors_2d.mean(axis=0)) / phrase_vectors_2d.std(axis=0) 


In [32]:
draw_vectors(phrase_vectors_2d[:, 0], phrase_vectors_2d[:, 1],
             phrase=[phrase[:50] for phrase in chosen_phrases],
             radius=20,)

figure(id='p1106', ...)

Наконец, давайте создадим простой движок для поиска "похожих вопросов" с использованием векторных представлений фраз, которые мы создали.

In [33]:
# compute vector embedding for all lines in data
data_vectors = np.array(list(map(lambda x: get_phrase_embedding(x, show_debug=False), data)))

In [34]:
def find_nearest(query: str, k: int = 10, get_sim: bool = False):
    """
    given text line (query), return k most similar lines from data, sorted from most to least similar
    similarity should be measured as cosine between query and line embedding vectors
    hint: it's okay to use global variables: data and data_vectors. see also: np.argpartition, np.argsort
    """
    # YOUR CODE
    query_vector: np.ndarray = get_phrase_embedding(query)
    
    similarities: np.ndarray = model.cosine_similarities(query_vector, vectors_all=data_vectors)
    
    # `nan` handling, `nan` should be shifted into out of similarity scale to left side 
    similarities[np.isnan(similarities)] = -1.
    
    top_k_partitioned = np.argpartition(similarities, -k)[-k:]
    print("[find_nearest] top_k_partitioned.shape: ", top_k_partitioned.shape)
    
    top_k_indices = top_k_partitioned[np.argsort(similarities[top_k_partitioned])[::-1]]
    print("[find_nearest] top_k_indices.shape: ", top_k_indices.shape)
    print("="*100)
    print("[find_nearest]: Founded top k with sim score: ", [(data[i], similarities[i]) for i in top_k_indices])
    print("="*100)
    
    if get_sim:
        top_k_results = [(data[i], similarities[i]) for i in top_k_indices]
    else:
        top_k_results = [data[i] for i in top_k_indices if not np.isnan(similarities[i])]

    return top_k_results

In [35]:
results = find_nearest(query="How do i enter the matrix?", k=10)
results

[get_phrase_embedding] Tokens before filter: 7
[get_phrase_embedding] Tokens after filter: 7
[find_nearest] top_k_partitioned.shape:  (10,)
[find_nearest] top_k_indices.shape:  (10,)
[find_nearest]: Founded top k with sim score:  [('How do I get to the dark web?\n', 0.980486965885441), ('What should I do to enter hollywood?\n', 0.9796027316247575), ('How do I use the Greenify app?\n', 0.9795014281666351), ('What can I do to save the world?\n', 0.979443766744798), ('How do I win this?\n', 0.9794301983223812), ('How do I think out of the box? How do I learn to think out of the box?\n', 0.9794210150875513), ('How do I find the 5th dimension?\n', 0.9792137082980186), ('How do I use the pad in MMA?\n', 0.9788570481310912), ('How do I estimate the competition?\n', 0.9786362513841781), ('What do I do to enter the line of event management?\n', 0.9781531814367285)]


/Users/dmitrii.koriakov/miniconda3/envs/ml-nlp-cv-lab-1/lib/python3.11/site-packages/gensim/models/keyedvectors.py:1162: RuntimeWarning: invalid value encountered in divide
  similarities = dot_products / (norm * all_norms)


['How do I get to the dark web?\n',
 'What should I do to enter hollywood?\n',
 'How do I use the Greenify app?\n',
 'What can I do to save the world?\n',
 'How do I win this?\n',
 'How do I think out of the box? How do I learn to think out of the box?\n',
 'How do I find the 5th dimension?\n',
 'How do I use the pad in MMA?\n',
 'How do I estimate the competition?\n',
 'What do I do to enter the line of event management?\n']

In [36]:
results = find_nearest(query="How do i enter the matrix?", k=10)

print(''.join(results))

assert len(results) == 10 and isinstance(results[0], str)
assert results[0] == 'How do I get to the dark web?\n'
assert results[3] == 'What can I do to save the world?\n'

[get_phrase_embedding] Tokens before filter: 7
[get_phrase_embedding] Tokens after filter: 7
[find_nearest] top_k_partitioned.shape:  (10,)
[find_nearest] top_k_indices.shape:  (10,)
[find_nearest]: Founded top k with sim score:  [('How do I get to the dark web?\n', 0.980486965885441), ('What should I do to enter hollywood?\n', 0.9796027316247575), ('How do I use the Greenify app?\n', 0.9795014281666351), ('What can I do to save the world?\n', 0.979443766744798), ('How do I win this?\n', 0.9794301983223812), ('How do I think out of the box? How do I learn to think out of the box?\n', 0.9794210150875513), ('How do I find the 5th dimension?\n', 0.9792137082980186), ('How do I use the pad in MMA?\n', 0.9788570481310912), ('How do I estimate the competition?\n', 0.9786362513841781), ('What do I do to enter the line of event management?\n', 0.9781531814367285)]
How do I get to the dark web?
What should I do to enter hollywood?
How do I use the Greenify app?
What can I do to save the world?


In [37]:
find_nearest(query="How does Trump?", k=10)

[get_phrase_embedding] Tokens before filter: 4
[get_phrase_embedding] Tokens after filter: 4
[find_nearest] top_k_partitioned.shape:  (10,)
[find_nearest] top_k_indices.shape:  (10,)
[find_nearest]: Founded top k with sim score:  [('What does Donald Trump think about Israel?\n', 0.9766546214445961), ('What books does Donald Trump like?\n', 0.973454425608238), ('What does Donald Trump think of India?\n', 0.973277918908444), ('What does India think of Donald Trump?\n', 0.9732779174611427), ('What does Donald Trump think of China?\n', 0.9724931538947921), ('What does Donald Trump think about Pakistan?\n', 0.9717841692863791), ('What companies does Donald Trump own?\n', 0.9715392722268258), ('What does Dushka Zapata think about Donald Trump?\n', 0.9714993657209735), ('How does it feel to date Ivanka Trump?\n', 0.9707504171143176), ('What does salesforce mean?\n', 0.9690142648198632)]


['What does Donald Trump think about Israel?\n',
 'What books does Donald Trump like?\n',
 'What does Donald Trump think of India?\n',
 'What does India think of Donald Trump?\n',
 'What does Donald Trump think of China?\n',
 'What does Donald Trump think about Pakistan?\n',
 'What companies does Donald Trump own?\n',
 'What does Dushka Zapata think about Donald Trump?\n',
 'How does it feel to date Ivanka Trump?\n',
 'What does salesforce mean?\n']

In [38]:
find_nearest(query="Why don't i ask a question myself?", k=10)

[get_phrase_embedding] Tokens before filter: 10
[get_phrase_embedding] Tokens after filter: 10
[find_nearest] top_k_partitioned.shape:  (10,)
[find_nearest] top_k_indices.shape:  (10,)
[find_nearest]: Founded top k with sim score:  [("Why don't I get a date?\n", 0.9832303064379523), ("Why do you always answer a question with a question? I don't, or do I?\n", 0.9820058043932623), ("Why can't I ask a question anonymously?\n", 0.9807827882535008), ("Why don't I get a girlfriend?\n", 0.9795394241404786), ("Why don't I have a boyfriend?\n", 0.9787765804022359), ("I don't have no question?\n", 0.9783484554457029), ("Why can't I take a joke?\n", 0.9782508376893226), ("Why don't I ever get a girl?\n", 0.9781665772649973), ("Can I ask a girl out that I don't know?\n", 0.9779957713776847), ("Why don't I have a girlfriend?\n", 0.9779327942853693)]


/Users/dmitrii.koriakov/miniconda3/envs/ml-nlp-cv-lab-1/lib/python3.11/site-packages/gensim/models/keyedvectors.py:1162: RuntimeWarning: invalid value encountered in divide
  similarities = dot_products / (norm * all_norms)


["Why don't I get a date?\n",
 "Why do you always answer a question with a question? I don't, or do I?\n",
 "Why can't I ask a question anonymously?\n",
 "Why don't I get a girlfriend?\n",
 "Why don't I have a boyfriend?\n",
 "I don't have no question?\n",
 "Why can't I take a joke?\n",
 "Why don't I ever get a girl?\n",
 "Can I ask a girl out that I don't know?\n",
 "Why don't I have a girlfriend?\n"]

##Multilingual Embedding-based Machine Translation (3 points)

**В этой ЛР** **<font color='red'>ТЫ</font>** создашь систему машинного перевода без использования параллельных корпусов, выравнивания, внимания, сверхкрутых рекуррентных нейронных сетей с 100500-уровневой глубиной и всех прочих наворочек

Но даже без параллельных корпусов эта система может быть достаточно хорошей (ну точно)

Для нашей системы мы выбираем два родственных славянских языка: украинский и русский.



### Почувствуй разницу!

(_синій кіт_ vs. _синій кит_)

![blue_cat_blue_whale.png](https://github.com/yandexdataschool/nlp_course/raw/master/resources/blue_cat_blue_whale.png)

### Фрагмент списка Swadesh для некоторых славянских языков

Список Swadesh - это лексико-статистический материал. Он назван в честь американского лингвиста Морриса Свадеша и содержит базовую лексику. Этот список используется для определения подгрупп языков, их родства.

Таким образом, мы можем наблюдать некую инвариантность слов для разных славянских языков.


| Russian         | Belorussian              | Ukrainian               | Polish             | Czech                         | Bulgarian            |
|-----------------|--------------------------|-------------------------|--------------------|-------------------------------|-----------------------|
| женщина         | жанчына, кабета, баба    | жінка                   | kobieta            | žena                          | жена                  |
| мужчина         | мужчына                  | чоловік, мужчина        | mężczyzna          | muž                           | мъж                   |
| человек         | чалавек                  | людина, чоловік         | człowiek           | člověk                        | човек                 |
| ребёнок, дитя   | дзіця, дзіцёнак, немаўля | дитина, дитя            | dziecko            | dítě                          | дете                  |
| жена            | жонка                    | дружина, жінка          | żona               | žena, manželka, choť          | съпруга, жена         |
| муж             | муж, гаспадар            | чоловiк, муж            | mąż                | muž, manžel, choť             | съпруг, мъж           |
| мать, мама      | маці, матка              | мати, матір, неня, мама | matka              | matka, máma, 'стар.' mateř    | майка                 |
| отец, тятя      | бацька, тата             | батько, тато, татусь    | ojciec             | otec                          | баща, татко           |
| много           | шмат, багата             | багато                  | wiele              | mnoho, hodně                  | много                 |
| несколько       | некалькі, колькі         | декілька, кілька        | kilka              | několik, pár, trocha          | няколко               |
| другой, иной    | іншы                     | інший                   | inny               | druhý, jiný                   | друг                  |
| зверь, животное | жывёла, звер, істота     | тварина, звір           | zwierzę            | zvíře                         | животно               |
| рыба            | рыба                     | риба                    | ryba               | ryba                          | риба                  |
| птица           | птушка                   | птах, птиця             | ptak               | pták                          | птица                 |
| собака, пёс     | сабака                   | собака, пес             | pies               | pes                           | куче, пес             |
| вошь            | вош                      | воша                    | wesz               | veš                           | въшка                 |
| змея, гад       | змяя                     | змія, гад               | wąż                | had                           | змия                  |
| червь, червяк   | чарвяк                   | хробак, черв'як         | robak              | červ                          | червей                |
| дерево          | дрэва                    | дерево                  | drzewo             | strom, dřevo                  | дърво                 |
| лес             | лес                      | ліс                     | las                | les                           | гора, лес             |
| палка           | кій, палка               | палиця                  | patyk, pręt, pałka | hůl, klacek, prut, kůl, pálka | палка, пръчка, бастун |

## Data

In [ ]:
import gensim
import numpy as np
from gensim.models import KeyedVectors

Скачивать эмбеды надо тут:
* [cc.uk.300.vec.zip](https://yadi.sk/d/9CAeNsJiInoyUA)
* [cc.ru.300.vec.zip](https://yadi.sk/d/3yG0-M4M8fypeQ)

Качаем эмбеды

In [ ]:
uk_emb = KeyedVectors.load_word2vec_format("cc.uk.300.vec")

In [ ]:
ru_emb = KeyedVectors.load_word2vec_format("cc.ru.300.vec")

In [ ]:
ru_emb.most_similar([ru_emb["август"]], topn=10)

In [ ]:
uk_emb.most_similar([uk_emb["серпень"]])

In [ ]:
ru_emb.most_similar([uk_emb["серпень"]])

Построить словари для тренировочной и тестовой выгрузки (да, да, будет)

А вот и файлы!

[ukr-rus-test](https://disk.yandex.ru/i/fXtdDXKwgJloCw)

[ukr-rus-train](https://disk.yandex.ru/d/lUy0sQZ1XBCchQ)

In [ ]:
def load_word_pairs(filename):
    uk_ru_pairs = []
    uk_vectors = []
    ru_vectors = []
    with open(filename, "r") as inpf:
        for line in inpf:
            uk, ru = line.rstrip().split("\t")
            if uk not in uk_emb or ru not in ru_emb:
                continue
            uk_ru_pairs.append((uk, ru))
            uk_vectors.append(uk_emb[uk])
            ru_vectors.append(ru_emb[ru])
    return uk_ru_pairs, np.array(uk_vectors), np.array(ru_vectors)

In [ ]:
uk_ru_train, X_train, Y_train = load_word_pairs("ukr_rus.train.txt")

In [ ]:
uk_ru_test, X_test, Y_test = load_word_pairs("ukr_rus.test.txt")

## Embedding space mapping

Мы рассматриваем задачу обучения линейного преобразования, которое минимизирует различие между векторными представлениями слов в исходном и целевом языках.

Пусть $x_i \in \mathbb{R}^d$ — векторное представление слова на исходном языке, а $y_i \in \mathbb{R}^d$ — вектор его перевода на целевой язык. Наша цель — найти матрицу $W$, которая минимизирует евклидово расстояние между преобразованными векторами $Wx_i$ и их соответствующими $y_i$.

Очень похоже на линейную регрессию, не так ли?

$$W^* = \arg\min_W \sum_{i=1}^n ||Wx_i - y_i||_2^2,$$


В мифологии Прокруст был известен тем, что «подгонял» людей под размеры своей кровати, растягивая их или отрезая им ноги, чтобы они идеально подходили. Подобным образом, в данной задаче мы «подгоняем» пространство встраивания исходного языка к пространству целевого языка, пытаясь свести их различия к минимуму.

![embedding_mapping.png](https://github.com/yandexdataschool/nlp_course/raw/master/resources/embedding_mapping.png)

![procrustes.png](https://github.com/yandexdataschool/nlp_course/raw/master/resources/procrustes.png)

Целевой язык - русский, исходный - украинский

In [ ]:
from sklearn.linear_model import LinearRegression

# YOUR CODE HERE

Давайте посмотрим на соседей вектора слова _«серпень»_ (_«август»_ по-русски) после линейного преобразования.

In [ ]:
august = mapping.predict(uk_emb["серпень"].reshape(1, -1))
ru_emb.most_similar(august)

Верный месяц всего на 9ом месте, хм...


В качестве меры качества мы будем использовать точность top@1, top@5 и top@10 (для каждого преобразованного украинского эмбеддинга мы подсчитываем, сколько правильных пар найдено в N ближайших соседях в пространстве русских эмбеддингов)

In [ ]:
def precision(pairs, mapped_vectors, topn=1):
    """
    :args:
        pairs = список пар правильных слов [(uk_word_0, ru_word_0), ...].
        mapped_vectors = список эмбедов после отображения из исходного пространства эмбедов в целевое пространство эмбедов
        topn = количество ближайших соседей в целевом пространстве эмбедов, из которых нужно выбрать
    """
    assert len(pairs) == len(mapped_vectors)
    num_matches = 0
    for i, (_, ru) in enumerate(pairs):
        # YOUR CODE HERE
    precision_val = num_matches / len(pairs)
    return precision_val


In [ ]:
assert precision([("серпень", "август")], august, topn=5) == 0.0
assert precision([("серпень", "август")], august, topn=9) == 1.0
assert precision([("серпень", "август")], august, topn=10) == 1.0

In [ ]:
assert precision(uk_ru_test, X_test) == 0.0
assert precision(uk_ru_test, Y_test) == 1.0

In [ ]:
precision_top1 = precision(uk_ru_test, mapping.predict(X_test), 1)
precision_top5 = precision(uk_ru_test, mapping.predict(X_test), 5)

assert precision_top1 >= 0.635
assert precision_top5 >= 0.813

## Making it better (orthogonal Procrustean problem)

Можно показать (см. оригинальную статью), что между семантическими пространствами должно быть ортогональным.

Мы можем ограничить преобразование $W$, чтобы оно было ортогональным. Тогда мы решим следующую задачу:

$$W^*= \arg\min_W ||WX - Y||_F \text{, где: } W^TW = I$$

$$I \text{- единичная матрица}$$.

Но ограничивать лосс в линейной регрессии как-то в падлу, поэтому мы можем найти оптимальное ортогональное преобразование с помощью разложения по сингулярным значениям (в чате тг я скидывал ссылки на краткое объяснение SVD). Оказывается, оптимальное преобразование $W^*$ может быть выражено через компоненты SVD:
$$X^TY=U\Sigma V^T\text{, singular value decompostion}$$.
$$W^*=UV^T$$

In [ ]:
def learn_transform(X_train, Y_train):
    """
    :returns: W* : float matrix[emb_dim x emb_dim] as defined in formulae above
    """
    # YOU CODE HERE

In [ ]:
W = learn_transform(X_train, Y_train)

In [ ]:
ru_emb.most_similar([np.matmul(uk_emb["серпень"], W)])

In [ ]:
assert precision(uk_ru_test, np.matmul(X_test, W)) >= 0.653
assert precision(uk_ru_test, np.matmul(X_test, W), 5) >= 0.824

## UK-RU Translator

Теперь мы готовы сделать простой переводчик на основе слов: для каждого слова исходного языка в общем пространстве вкраплений мы находим ближайшее в языке перевода.

[fairy_tale.txt](https://disk.yandex.ru/d/4P9xWLfm-GnauQ) - качаем

In [ ]:
with open("fairy_tale.txt", "r") as inpf:
    uk_sentences = [line.rstrip().lower() for line in inpf]

In [ ]:
def translate(sentence):
    """
    :args:
        sentence - предложение на украинском языке (str)
    :returns:
        перевод - предложение на русском языке (str)

    * найти украинские эмбеды для каждого слова в предложении
    * преобразовать вектор украинских эмбедов
    * найти ближайшее русское слово и заменить на него
    """
    # YOUR CODE HERE

In [ ]:
assert translate(".") == "."
assert translate("1 , 3") == "1 , 3"
assert translate("кіт зловив мишу") == "кот поймал мышку"

In [ ]:
for sentence in uk_sentences:
    print("src: {}\ndst: {}\n".format(sentence, translate(sentence)))

Очень неплохо, но как можно еще улучшить результат?

## А теперь самое интересное?

### Почитать:
* [Exploiting Similarities among Languages for Machine Translation](https://arxiv.org/pdf/1309.4168)  - entry point for multilingual embedding studies by Tomas Mikolov (the author of W2V)
* [Offline bilingual word vectors, orthogonal transformations and the inverted softmax](https://arxiv.org/pdf/1702.03859) - orthogonal transform for unsupervised MT
* [Word Translation Without Parallel Data](https://arxiv.org/pdf/1710.04087)
* [Loss in Translation: Learning Bilingual Word Mapping with a Retrieval Criterion](https://arxiv.org/pdf/1804.07745)
* [Unsupervised Alignment of Embeddings with Wasserstein Procrustes](https://arxiv.org/pdf/1805.11222)
